In [1]:
# Instala o findspark
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import findspark
findspark.init()
import os

import pyspark.sql.functions as F
from pyspark.sql.types import *

## Atention to the version
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'

In [3]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("projeto").getOrCreate()
## Create a spark session
# spark = SparkSession.builder.appName("SparkDemo").getOrCreate()
spark = (SparkSession
         .builder
         .appName("SparkDemo")
         .master("local[*]")      
         .getOrCreate()
)
         # .config("spark.executor.memory", "8G")
         # .config("spark.executoar.pysprk.memory", "8G")
         # .config("spark.cores.max", "3")
         # .config("spark.scheduler.listenerbus.eventqueue.capacity", "8")
         # .config("spark.sql.shuffle.partitions", "200")
         # .config("spark.sql.streaming.checkpointLocation", "./check.txt")
         # .config("spark.streaming.receiver.maxRate", "10")
         # .config("spark.log.level", "DEBUG")  # ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN

         # .builder
         # .config("spark.app.name", "SparkDemo")
         # .master("local[*]")         
         # .getOrCreate()

24/05/24 17:58:21 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/05/24 17:58:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cj/.ivy2/cache
The jars for the packages stored in: /home/cj/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-31e8d620-aa71-4f31-83c2-78269938b404;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 509ms :: artifacts dl 17ms
	:: modules

In [4]:
# spark.sparkContext.setLogLevel('ERROR')
# ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN
# param = spark.sparkContext._conf.get("spark.sql.streaming.continuous.epochBacklogQueueSize")
# print(param)

In [5]:
## Input Schema
df_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("transaction_timestamp", TimestampType(), True),
    StructField("merchant_id", StringType(), True)
])

In [6]:
# ## Kafka configs
# kafka_input_config = {
#     "kafka.bootstrap.servers" : "localhost:9092",
#     "subscribe" : "vandas-deshboard-bronze",
#     "startingOffsets" : "latest",
#     "failOnDataLoss" : "false"
#     # "min.insync.replicas": 2,
#     # "processing.guarantee":"exactly_once",
#     # "EnableAutoCommit" : "true",
#     # "AutoCommitIntervalMs" : 5000,
#     # "EnableAutoOffsetStore" : "true"
# }
# ,
#     "startingOffsets" : "latest",
#     "failOnDataLoss" : "false"

## Read Stream
df = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", "vandas-deshboard-bronze")\
    .option("startingOffsets", "latest")\
    .load()\
    .selectExpr("CAST(value AS STRING)")\
    .withColumn("jsonData", F.from_json(F.col("value"), df_schema)).select("jsonData.*")
df

# \
#     .select(F.from_json(F.col("value").cast("string"),df_schema))

    # .trigger(Trigger.ProcessingTime("5 seconds"))\


DataFrame[transaction_id: string, customer_id: string, amount: int, transaction_timestamp: timestamp, merchant_id: string]

In [7]:
## Filter for eligible transactions
df = df.filter(\
              (F.col("amount") >= 15) &\
              (F.dayofweek(F.col("transaction_timestamp")) == 6 ) &\
              (F.col("merchant_id") == "MerchantX")\
              )\
        .withColumn("cashback",F.col("amount").cast("double") * 0.15 )\
        .select(F.col("customer_id"),F.col("amount"),\
                F.col("transaction_timestamp"),F.col("merchant_id"),F.col("cashback"))
df
output_df = df.select(F.to_json(F.struct(*df.columns)).alias("value"))

In [8]:
# spark.sparkContext.setLogLevel('ERROR')
# ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN

In [9]:
kafka_output_config = {
    "kafka.bootstrap.servers" : "localhost:9092",
    "topic" : "vandas-deshboard-gold",
    "checkpointLocation" : "./check2.txt",
    # "min.insync.replicas": 2,
    # "processing.guarantee":"exactly_once",
    # "EnableAutoCommit" : "true",
    # "AutoCommitIntervalMs" : 5000,
    # "EnableAutoOffsetStore" : "true"
}
# Write key-value data from a DataFrame to a specific Kafka topic specified in an option
ds = output_df \
    .selectExpr("CAST(value AS STRING)") \
    .writeStream \
    .format("kafka") \
    .options(**kafka_output_config)\
    .start()

ds.awaitTermination()

24/05/24 17:58:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/05/24 17:58:27 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Transaction Sample
{"transaction_id": "1", "customer_id": "123", "amount": 20, "transaction_timestamp": "2023-04-21T09:30:001Z", "merchant_id": "MerchantX"}
{"transaction_id": "2", "customer_id": "23", "amount": 20, "transaction_timestamp": "2023-04-21T09:30:00Z", "merchant_id": "MerchantY"}
{"transaction_id": "3", "customer_id": "12", "amount": 20, "transaction_timestamp": "2023-04-22T09:30:00Z", "merchant_id": "MerchantX"}
{"transaction_id": "4", "customer_id": "321", "amount": 667, "transaction_timestamp": "2023-04-28T09:30:00Z", "merchant_id": "MerchantX"}
